In [29]:
%cd /Users/trancatkhanh/Downloads/GAME
from base.Splendor_v3.env import *


/Users/trancatkhanh/Downloads/GAME


# Homework 3:
**1 agent tổng quán sử dụng bộ feature và bias có tỷ lệ thắng trên 60%**

# Bước 1 :Lấy bias của các action

In [2]:
from base.Splendor_v3.env import *
import numpy as np
bias1 = [np.random.rand(getActionSize()),np.zeros(getActionSize())]
@njit()
def p_bias(state,per):
    actions = getValidActions(state)
    actions *= per[0]
    action = np.argmax(actions)
    if getReward(state) == 1:
        per[1] += per[0]
    if getReward(state) == 0:
        per[0] = np.random.rand(getActionSize())
    return action,per

win, bias = numba_main_2(p_bias, 100000,bias1,0)
# Bias tối ưu 
bias_greedy = (bias[1]/np.max(bias[1]))
print(bias_greedy)
# Check tỷ lệ thắng của bias tối ưu 
@njit()
def p_bias_optim(state,per):
    actions = getValidActions(state)
    actions *= bias_greedy
    action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(p_bias_optim, 1000,bias1,0)
print(win)

[0.93806739 0.95570509 0.96912988 1.         0.89858925 0.89941295
 0.90133308 0.90615644 0.90167387 0.91297777 0.90765594 0.90910291
 0.9097714  0.91116202 0.90296255]
562


# Bước 2: Ghép các feature với nhau 

# Cách 1

In [ ]:
def get_feature5(i,old,new):
  #State old là state Hiện tại 
  #State new là state Mới 
  rate = 0 
  while rate<0.2 :

    # state mà có cả Value của old and new (1,0)
    o = (new>old)
    o= 1*o
    i = np.zeros((old.shape[0],1))
    for u in range(old.shape[0]) :
        if sum(o[u])>0 :
            # print(o[u])
            i[u] = 1
    #  a matrix with random value
    l = np.random.uniform(-1,1,(161,5))
    for j in range(getStateSize()):
      for vl in range(0,5):
        l[j][vl] = np.random.choice([-1,0,1], 1, p=[0.05, 0.75, 0.2])

        # if l[j][vl] > np.random.randn(1):
        #     l[j][vl] = 1
        # elif l[j][vl] < np.random.randn(1):
        #     l[j][vl] =  0
        # else:
        #     l[j][vl] = -1
    #state mà kích hoạt (1,0)
    
    k = 1*((new@l)>=0)
    # state mà kích hoạt và có reward (1,0)
    y = np.zeros((old.shape[0],1))
    for p in range(old.shape[0]):
        if sum(k[p]) >= 1 and sum(i[p])>= 1:
            y[p] = 1
        else:
            y[p] = 0
      
    rate = (np.sum(y)/np.sum(k))

  return l.reshape(5,161)
matrix_feature_check_win_rate = []
for k in range(0,15): 
  old = np.load('old'+f'{k}'+'.npy')
  new = np.load('new'+f'{k}'+'.npy')
  matrix  = get_feature5(k,old,new)
  matrix_feature = []
  matrix_feature.append(matrix)
  perx=[np.zeros((5,getStateSize())),np.zeros((1,5))]
  for i in range(0,5):
      for j in range(getStateSize()):
          perx[0][i][j] += matrix_feature[0][0][j]
  @njit
  def p0(state,per):
      actions = getValidActions(state)
      matran1 = per[0]@state    
      matran1 = 1*(matran1 >= 0)
      actions = np.where(actions == 1)[0]
      if np.sum(matran1) == 5 and k in actions :
          action = k
      else:
          actions = np.where(actions == 1)[0]
          action = actions[np.random.randint(len(actions))]
      return action,per
  win, x = numba_main_2(p0, 300,perx,0)
  while  win<350/1000*300 :
    matrix  = get_feature5(k,old,new)
    matrix_feature = []
    matrix_feature.append(matrix)
    perx=[np.zeros((5,getStateSize())),np.zeros((1,5))]
    for i in range(0,5):
        for j in range(getStateSize()):
            perx[0][i][j] += matrix_feature[0][0][j]
    win, x = numba_main_2(p0, 300,perx,0)
    print('phần trăm trận thắng của action '+f'{k}',win/300)

  print('phần trăm trận thắng của action '+f'{k}',win/300)

  matrix_feature_check_win_rate.append(matrix)

np.save('feature.npy',matrix_feature_check_win_rate)
perx=[np.zeros((75,getStateSize())),np.zeros((getActionSize(),75))]
for vk in range(0,15):
  for i in range(0,5):
      for j in range(getStateSize()):
          perx[0][vk*5+i][j] = matrix_feature_check_win_rate[vk][i][j]
          
for i in range(0,15):
    for j in range(5):
        perx[1][i][i*5+j] = 1
        
from base.Splendor_v3.env import *
@njit
def p0(state,per):
    actions = getValidActions(state)
    matran1 = per[0]@state
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)    
    matran2 = per[1]@matran1
    matran2 *= actions
    if len(np.where(matran2 == matran2[np.argmax(matran2)])[0])>1 and np.max(matran2) == 5:
        matran2 += bias_greedy
        action = np.argmax(matran2)
    else:        
        matran2 += bias_greedy
        action = np.argmax(matran2)
    return action,per
win, x = numba_main_2(p0, 1000,perx,0)
print(win)

# Cách 2

In [32]:
def get_feature5(old,new):
  #State old là state Hiện tại 
  #State new là state Mới 
  rate = 0 
  while rate<0.2 :

    # state mà có cả Value của old and new (1,0)
    o = (new>old)
    o= 1*o
    i = np.zeros((old.shape[0],1))
    for u in range(old.shape[0]) :
        if sum(o[u])>0 :
            # print(o[u])
            i[u] = 1
    #  a matrix with random value
    l = np.zeros((5,getStateSize()))
    for ii in range(0,5):
        l[ii][np.random.randint(getStateSize())] += 1
        l[ii][np.random.randint(getStateSize())] += 1
        l[ii][np.random.randint(getStateSize())] += -1
    l=l.reshape(161,5)
    #state mà kích hoạt (1,0)
    
    k = 1*((new@l)>=0)
    # state mà kích hoạt và có reward (1,0)
    y = np.zeros((old.shape[0],1))
    for p in range(old.shape[0]):
        if sum(k[p]) >= 1 and sum(i[p])>= 1:
            y[p] = 1
        else:
            y[p] = 0
      
    rate = (np.sum(y)/np.sum(k))

  return l.reshape(5,161)
# matrix_feature_check_win_rate = []
# for k in range(0,15): 
#   old = np.load('old'+f'{k}'+'.npy')
#   new = np.load('new'+f'{k}'+'.npy')
#   matrix  = get_feature5(old,new)
#   matrix_feature = []
#   matrix_feature.append(matrix)
#   perx=[np.zeros((5,getStateSize())),np.zeros((1,5))]
#   for i in range(0,5):
#       for j in range(getStateSize()):
#           perx[0][i][j] += matrix_feature[0][0][j]
#   @njit
#   def p0(state,per):
#       actions = getValidActions(state)
#       matran1 = per[0]@state    
#       matran1 = 1*(matran1 >= 0)
#       actions = np.where(actions == 1)[0]
#       if np.sum(matran1) == 5 and k in actions :
#           action = k
#       else:
#           actions = np.where(actions == 1)[0]
#           action = actions[np.random.randint(len(actions))]
#       return action,per
#   win, x = numba_main_2(p0, 300,perx,0)
#   while  win<350/1000*300 :
#     matrix  = get_feature5(old,new)
#     matrix_feature = []
#     matrix_feature.append(matrix)
#     perx=[np.zeros((5,getStateSize())),np.zeros((1,5))]
#     for i in range(0,5):
#         for j in range(getStateSize()):
#             perx[0][i][j] += matrix_feature[0][0][j]
#     win, x = numba_main_2(p0, 300,perx,0)
#     print('phần trăm trận thắng của action '+f'{k}',win/300)

#   print('phần trăm trận thắng của action '+f'{k}',win/300)

#   matrix_feature_check_win_rate.append(matrix)


# Cách 3

In [38]:
while win<600:
  matrix_feature_check_win_rate = []
  for k in range(0,15): 
    old = np.load('old'+f'{k}'+'.npy')
    new = np.load('new'+f'{k}'+'.npy')
    matrix  = get_feature5(old,new)
    matrix_feature = []
    matrix_feature.append(matrix)
    perx=[np.zeros((5,getStateSize())),np.zeros((1,5))]
    for i in range(0,5):
        for j in range(getStateSize()):
            perx[0][i][j] += matrix_feature[0][0][j]
    @njit
    def p0(state,per):
        actions = getValidActions(state)
        matran1 = per[0]@state    
        matran1 = 1*(matran1 >= 0)
        actions = np.where(actions == 1)[0]
        if np.sum(matran1) == 5 and k in actions :
            action = k
        else:
            actions = np.where(actions == 1)[0]
            action = actions[np.random.randint(len(actions))]
        return action,per
    win, x = numba_main_2(p0, 1000,perx,0)
    while  win<400 :
      matrix  = get_feature5(old,new)
      matrix_feature = []
      matrix_feature.append(matrix)
      perx=[np.zeros((5,getStateSize())),np.zeros((1,5))]
      for i in range(0,5):
          for j in range(getStateSize()):
              perx[0][i][j] += matrix_feature[0][0][j]
      win, x = numba_main_2(p0, 1000,perx,0)
      print('phần trăm trận thắng của action '+f'{k}',win/1000)

    print('phần trăm trận thắng của action '+f'{k}',win/1000)

    matrix_feature_check_win_rate.append(matrix)

  # listy = np.arange(15)
  # np.random.shuffle(listy)
  perx=[np.zeros((75,getStateSize())),np.zeros((getActionSize(),75))]

  for vk in range(0,15):
    for i in range(0,5):
        for j in range(getStateSize()):
            perx[0][vk*5+i][j] = matrix_feature_check_win_rate[vk][i][j]
            
  for i in range(0,15):
      for j in range(5):
          perx[1][i][i*5+j] = 1
  from base.Splendor_v3.env import *
  @njit
  def p01(state,per):
      actions = getValidActions(state)
      matran1 = per[0]@state
      for i in range(len(matran1)):
          matran1[i] = int(matran1[i] >= 0)    
      matran2 = per[1]@matran1
      matran2 *= actions
      matran2 += bias_greedy
      action = np.argmax(matran2)
      return action,per
  win, x = numba_main_2(p01, 1000,perx,0)
  print('số trận thắng để đạt 60%',win)
print('số trận thắng  60%',win)

phần trăm trận thắng của action 0 0.428
phần trăm trận thắng của action 1 0.491
phần trăm trận thắng của action 2 0.504
phần trăm trận thắng của action 3 0.51
phần trăm trận thắng của action 4 0.364
phần trăm trận thắng của action 4 0.364
phần trăm trận thắng của action 5 0.277
phần trăm trận thắng của action 5 0.336
phần trăm trận thắng của action 5 0.304
phần trăm trận thắng của action 5 0.31
phần trăm trận thắng của action 5 0.428
phần trăm trận thắng của action 5 0.428
phần trăm trận thắng của action 6 0.329
phần trăm trận thắng của action 6 0.266
phần trăm trận thắng của action 6 0.335
phần trăm trận thắng của action 6 0.339
phần trăm trận thắng của action 6 0.299
phần trăm trận thắng của action 6 0.305
phần trăm trận thắng của action 6 0.317
phần trăm trận thắng của action 6 0.298
phần trăm trận thắng của action 6 0.294
phần trăm trận thắng của action 6 0.291
phần trăm trận thắng của action 6 0.348
phần trăm trận thắng của action 6 0.336
phần trăm trận thắng của action 6 0.361
ph

KeyboardInterrupt: 

# Cách 4

In [8]:
import numpy as np
from numba import njit 
#Khởi tạo weight 
perx = [np.zeros((75,getStateSize())),np.zeros((15,75)),np.zeros((2,1))]
for i in range(0,15):
    for j in range(5):
        perx[0][i*5+j][np.random.randint(getStateSize())] = 1
        perx[0][i*5+j][np.random.randint(getStateSize())] = 1
        perx[0][i*5+j][np.random.randint(getStateSize())] = -1
for i in range(0,15):
    for j in range(5):
        perx[1][i][i*5+j] = 1
from base.Splendor_v3.env import *
@njit
def p0(state,per):
    if per[2][1]>20:
        if per[2][0]/per[2][1] < 0.6:
            per[2][0][0] = 0  
            per[2][0][1] = 0
            for i in range(0,15):
                for j in range(5):
                    for k in range(getStateSize()):
                        per[0][i*5+j][k] = 0
            for i in range(0,15):
                for j in range(5):
                    per[0][i*5+j][np.random.randint(getStateSize())] = 1
                    per[0][i*5+j][np.random.randint(getStateSize())] = 1
                    per[0][i*5+j][np.random.randint(getStateSize())] = -1   
    actions = getValidActions(state)
    matran1 = per[0]@state
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)    
    matran2 = per[1]@matran1
    matran2 *= actions
    if len(np.where(matran2 == matran2[np.argmax(matran2)])[0])>1 and np.max(matran2) == 5:
        matran2 += bias_greedy
        action = np.argmax(matran2)
    else:        
        action = np.argmax(matran2)
    if getReward(state) == 1:
        per[2][0] += 1  
        per[2][1] += 1
    if getReward(state) == 0:
        per[2][1] += 1
    return action,per
win, x = numba_main_2(p0, 300000,perx,0)
print(win)


126846


In [27]:
print(perx[0][1])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [15]:
from base.Splendor_v3.env import *
@njit
def p1(state,per):
    actions = getValidActions(state)
    matran1 = per[0]@state
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)    
    matran2 = per[1]@matran1
    matran2 *= actions
    if len(np.where(matran2 == matran2[np.argmax(matran2)])[0])>1 and np.max(matran2) == 5:
        matran2 += bias_greedy
        action = np.argmax(matran2)
    else:        
        action = np.argmax(matran2)
    return action,per
win, x1 = numba_main_2(p1, 1000,x,0)
print(win)

375
